# Import Modules

In [1]:
import numpy as np
import pandas as pd

import utils

# Load Data

In [8]:
target_vegetable = utils.VEGETABLES[0]

In [9]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

# train_test["weekday"] = train_test.date.dt.weekday
# weekdays = pd.get_dummies(train_test["weekday"])
# train_test = train_test.drop(columns="weekday")
# train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

train_loader, train, test, ss = utils.preprocess_data_submit(train, test)

# Set Free Params

In [10]:
future = test.shape[0]
num_epochs = 70
learning_rate = 0.001
weight_decay = 1e-3

# Training

In [11]:
pred_y = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs,
                                   lr=learning_rate, weight_decay=weight_decay)

pred_y = pred_y.cpu().detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :6] = ss.inverse_transform(pred_y[:, :6])

In [12]:
print(f"std: {np.std(pred_y[:, 0].tolist())}")
pred_y[:, 0].tolist()

std: 78.0559964144987


[932.3714353363436,
 977.9071794429805,
 1035.6055467063386,
 1091.2644172339428,
 1133.8079732154406,
 1164.2550574811305,
 1184.5297035910276,
 1196.0461746464205,
 1209.8007218891378,
 1213.3830065954708,
 1211.4123987681694,
 1206.241390639571,
 1199.6221721779086,
 1192.781989375747,
 1186.5024012892968,
 1181.211304608772,
 1177.074765047395,
 1174.9178074651231,
 1181.6191982482922,
 1183.7622292114243]